# Table of Contents

1.  [ZIO](#org3312281)
    1.  [Life after Typeclasses](#orge41c864)
    2.  [In JDG&rsquo;s words](#org6f4f7ba)
    3.  [A Functional Effect](#orgd9ef2f5)
        1.  [An immutable, type-safe, tree-like data structure that models procedural effects.](#org80bb473)
        2.  [ReaderT Design Pattern](#orgb99ac5e)
        3.  [As Effect Wrapper](#org3419025)
        4.  [As Concurrency Framework](#orgab1c330)
2.  [ZLayer](#orgf36bf9a)
    1.  [`Has` is a type level map.](#org80935e9)
    2.  [How to create them](#org145560c)
    3.  [A ZLayer that takes input](#org7d9a2e0)
3.  [zio-test](#orge161833)
    1.  [Intro](#org47c1454)
    2.  [TestClock](#org3275b26)
4.  [ZIO Lock-in](#org6360a37)


<a id="org3312281"></a>

# ZIO


<a id="orge41c864"></a>

## Life after Typeclasses


* What to do with no monad-support
* What would Scala do?
* [In JDG&rsquo;s words](https://degoes.net/articles/zio-history)

<a id="orgd9ef2f5"></a>

## A Functional Effect


<a id="org80bb473"></a>

### An immutable, type-safe, tree-like data structure that models procedural effects.

    ZIO[R, E, A] ~> R => Either[E, A]

Except `R` now has `Layer` s and uses the `Has` typeclass trick


<a id="orgb99ac5e"></a>

### [ReaderT Design Pattern](https://www.fpcomplete.com/blog/2017/06/readert-design-pattern)


In [ ]:
import $ivy.`io.prometheus:simpleclient:0.8.1`
import $ivy.`io.prometheus:simpleclient_common:0.8.1`
import $ivy.`dev.zio::zio:1.0.0-RC18-2`

<a id="org419025"></a>

### As Effect Wrapper

Let&rsquo;s use the Prometheus Metrics Library as an example. Using this Java library in Scala looks like this:

In [ ]:
// This is a Java Library
import io.prometheus.client.CollectorRegistry
import io.prometheus.client.{ Counter, Histogram }
import io.prometheus.client.exporter.common.TextFormat

CollectorRegistry.defaultRegistry.clear()

// Spread operator => :_*
val c = Counter
    .build()
    .name("PrometheusCounter")
    .labelNames(Array("class", "method"): _*)
    .help(s"Sample prometheus counter")
    .register()

val h = Histogram
    .build()
    .name("PrometheusHistogram")
    .labelNames(Array("class", "method"): _*)
    .linearBuckets(0.0, 0.2, 5)
    .help(s"Sample prometheus histogram")
    .register()

`register()` without arguments uses the default `Collectorregistry`.
If we declare **labels** on building the metrics, then we ****NEED**** to pass the **\*exact same number of labels** when using the respective metric.





In [ ]:
c.labels(Array("Users", "getTotal"):_*).inc()
c.labels(Array("Users", "getUsers"):_*).inc(3)

h.labels(Array("Users", "getTotal"):_*).observe(0.32)
h.labels(Array("Users", "getUsers"):_*).observe(0.45)

To test its usage we can write a simple function that writes the **Registry** in the correct Prometheus format:

In [ ]:
import java.io.StringWriter

def write004(registry: CollectorRegistry) = {
    val writer = new StringWriter
    TextFormat.write004(writer, registry.metricFamilySamples)
    writer.toString
}

write004(CollectorRegistry.defaultRegistry)

Let&rsquo;s wrap this function and ask for the default registry in ZIO effects:

In [ ]:
import zio.console.putStrLn
import zio.console.Console
import zio.Runtime
import zio.Task

val writeT004: CollectorRegistry => Task[String] = registry => Task {
    val writer = new StringWriter
    TextFormat.write004(writer, registry.metricFamilySamples)
    writer.toString
}

val externalRegistry = Task.effect(CollectorRegistry.defaultRegistry)

val program = for {
    r <- externalRegistry
    s <- writeT004(r)
} yield s

val rt = Runtime.unsafeFromLayer(Console.live)

rt.unsafeRun(program >>= (s => putStrLn(s)))

-   Calling `Task` directly is the same as calling `Task.effect`
-   To run this effect we need a `Runtime` capable of writing to the **Console**


<a id="orgab1c330"></a>

### As Concurrency Framework

-   ZIO offers many utilities for concurrency such as `Fiber`, `Queue`, `Scheduler`, etc.
-   Let&rsquo;s define some basic functionality

In [ ]:
import zio.{Queue, UIO, URIO}

val queue: UIO[Queue[String]] = Queue.bounded[String](10)

def send(msg: String)(q: Queue[String]): URIO[Console, Unit] =
  for {
    _ <- putStrLn(s"Sending... $msg")
    _ <- q.offer(msg)
  } yield ()

def sendAsync(msg: String)(q: Queue[String]): UIO[Unit] =
  for {
    _ <- q.offer(msg).fork
  } yield ()

def listen(q: Queue[String]): URIO[Console, Unit] =
  for {
      s <- q.poll
      _ <- putStrLn(s"Received $s")
  } yield ()

-   `UIO` is a `ZIO[Any, Nothing, A]`, which means an effect that needs no environment and can&rsquo;t fail.
-   `send` just offers messages to the queue synchronously while `sendAsync` does so asynchronously
-   Important is to not that you need to pass the `queue` they will send messages to

Lets test we can send synchronously:

In [ ]:
val prog0 = for {
    q <- queue
    _ <- URIO.foreach(1 to 20)(i => send(s"Number $i")(q))
} yield ()

rt.unsafeRun(prog0)

Now lets send asynchronously, this time we will `listen` for the one message:

In [ ]:
val prog1 = for {
    q <- queue
    _ <- URIO.foreach(1 to 20)(i => sendAsync(s"Number $i")(q))
    _ <- listen(q)
} yield ()

rt.unsafeRun(prog1)

Now lets create a `scheduler` that polls until there are no more messages in the queue

In [ ]:
import zio.Schedule

val untilNothing = Schedule.doUntil[String](_ == "Nothing")

def listen(q: Queue[String]): URIO[Console, String] = 
  for {
      s <- q.poll
      _ <- putStrLn(s"Received $s")
  } yield s.getOrElse("Nothing")

val prog2 = for {
    q <- queue
    _ <- URIO.foreach(1 to 20)(i => sendAsync(s"Number $i")(q))
    l <- listen(q).repeat(untilNothing)
} yield l

rt.unsafeRun(prog2 >>= (s => putStrLn(s)))

Lets modify `listen` so that it collects messages, we applu an important principle of Pure FP in Scala: **make an unsafe but performant implementation but expose only a pure API**

In [ ]:
import zio.RIO

val untilNCollected = Schedule.doUntil[List[String]](_.size == 5)

def listen(q: Queue[String]): RIO[Console, List[String]] = {
  import scala.collection.mutable.ListBuffer
  var collector = new ListBuffer[String]()
  for {
      s <- q.poll
      _ <- putStrLn(s"Received $s")
      if (s.isDefined)
  } yield (collector += s.get).toList // safe cuz we filter on isDefined
}

val prog3 = for {
    q <- queue
    _ <- URIO.foreach(1 to 20)(i => sendAsync(s"Number $i")(q))
    l <- listen(q).repeat(untilNCollected)
} yield l

rt.unsafeRun(prog3 >>= (l => RIO.foreach(l)(s => putStrLn(s"Result: $s"))))

Since this version is **unsafe**, we switch `URIO` for its `throwable` counterpart: `RIO`.
Now a version that polls every 2 seconds

In [ ]:
import zio.duration._

val everyNSec = Schedule.spaced(2.seconds)
val prog4 = for {
    q <- queue
    _ <- URIO.foreach(1 to 20)(i => sendAsync(s"Number $i")(q))
    l <- listen(q).repeat(everyNSec)
} yield l

rt.unsafeRun(prog4.provideSomeLayer[Console](zio.clock.Clock.live))

Let&rsquo;s combine both schedulers and prepare a `Runtime` that has all effects we&rsquo;re gonna need:

In [ ]:
import zio.clock.Clock
import zio.duration._

val everyNSec = Schedule.spaced(2.seconds)

val rt = Runtime.unsafeFromLayer(Console.live ++ Clock.live)

val everyNSecUntilCollected = untilNCollected && everyNSec

val prog5 = for {
    q <- queue
    _ <- URIO.foreach(1 to 20)(i => sendAsync(s"Number $i")(q))
    l <- listen(q).repeat(everyNSecUntilCollected)
} yield l

rt.unsafeRun(prog5)

Finally, lets execute 2 effects in parallel, returning the result of the first

In [ ]:
def listen(q: Queue[String]): RIO[Console, List[String]] = {
  import scala.collection.mutable.ListBuffer
  var collector = new ListBuffer[String]()
  for {
      s <- q.poll
      _ <- putStrLn(s"Received $s")
      if (s.isDefined)
  } yield {
      if (collector.size == 5) collector.clear()
      (collector += s.get).toList
  }
}

val prog6 = for {
    q <- queue
    _ <- URIO.foreach(1 to 20)(i => sendAsync(s"Number $i")(q))
    l <- listen(q).repeat(untilNCollected) <& listen(q).repeat(everyNSec)
} yield l

rt.unsafeRun(prog6)

<a id="orgf36bf9a"></a>

# ZLayer


<a id="org80935e9"></a>

## `Has` is a type level map.


<a id="org145560c"></a>

## How to create them

-   `ZLayer.succeeds` created a layer that has no dependencies
-   `Metrics` is both the name of the **Layer** andat the type-level, its the name of a `Metrics.Service` **Has**.

In [ ]:
import zio.{ Has, Layer, ZLayer }
import zio.{ RIO, Task }

type Metrics = Has[Metrics.Service]

object Metrics {
    trait Service {
      def getRegistry(): Task[CollectorRegistry]

      def inc(tags: Array[String]): Task[Unit]

      def inc(amount: Double, tags: Array[String]): Task[Unit]

      def time(f: () => Unit, tags: Array[String]): Task[Double]

    }

    val live: Layer[Nothing, Metrics] = ZLayer.succeed(new Service {

      private val (myCounter, myHistogram) = {
          val c = Counter
            .build()
            .name("LayerCounter")
            .labelNames(Array("class", "method"): _*)
            .help(s"Prometheus counter inside a layer")
            .register()

        val h = Histogram
            .build()
            .name("LayerHistogram")
            .labelNames(Array("class", "method"): _*)
            .linearBuckets(0.0, 0.2, 5)
            .help(s"Prometheus histogram inside a layer")
            .register()
          
          (c, h)
      }

      def getRegistry(): Task[CollectorRegistry] =
        Task.effect(CollectorRegistry.defaultRegistry)

      def inc(tags: Array[String]): zio.Task[Unit] =
        inc(1.0, tags)

      def inc(amount: Double, tags: Array[String]): Task[Unit] =
        Task(myCounter.labels(tags:_*)inc(amount))

      def time(f: () => Unit, tags: Array[String]): Task[Double] = Task {
          val t = myHistogram.labels(tags: _*).startTimer()
          f()
          t.observeDuration()
      }
    })
}

Let&rsquo;s write a small program that uses:

In [ ]:
val program1 = for {
      m  <- RIO.environment[Metrics]
      _  <- putStrLn("Metrics")
      r  <- m.get.getRegistry()
      _  <- m.get.inc(Array("RequestCounter", "get"))
      _  <- m.get.inc(Array("RequestCounter", "post"))
      _  <- m.get.inc(2.0, Array("LoginCounter", "login"))
      _  <- m.get.time(() => Thread.sleep(2000), Array("histogram", "get"))
      s  <- writeT004(r)
      _  <- putStrLn(s)
    } yield ()

The type of `program` is `ZIO[Metrics with Console, Throwable, Unit]`. We can use the current **runtime** and just **provide** the extra layer:

In [ ]:
//CollectorRegistry.defaultRegistry.clear()
rt.unsafeRun(program1.provideSomeLayer[Console](Metrics.live))

Unlike `map`, the type parameter for `provideSomeLayer` is the ****output**** type.


<a id="org7d9a2e0"></a>

## A ZLayer that takes input

To Save space, let&rsquo;s write 2 versions of that do the same un der `Metrics2`:

In [ ]:
type Metrics2 = Has[Metrics2.Service]

object Metrics2 {
    trait Service {
      def getRegistry(): Task[CollectorRegistry]

      def inc(tags: Array[String]): Task[Unit]

      def inc(amount: Double, tags: Array[String]): Task[Unit]

      def time(f: () => Unit, tags: Array[String]): Task[Double]

    }

    val receiver: (Counter, Histogram) => Layer[Nothing, Metrics2] = 
    (myCounter, myHistogram) => ZLayer.succeed(new Service {

      def getRegistry(): Task[CollectorRegistry] =
        Task.effect(CollectorRegistry.defaultRegistry)

      def inc(tags: Array[String]): zio.Task[Unit] =
        inc(1.0, tags)

      def inc(amount: Double, tags: Array[String]): Task[Unit] =
        Task(myCounter.labels(tags:_*)inc(amount))

      def time(f: () => Unit, tags: Array[String]): Task[Double] = Task {
          val t = myHistogram.labels(tags: _*).startTimer()
          f()
          t.observeDuration()
      }
    })
    
    val receiverHas: ZLayer[Has[(Counter, Histogram)], Nothing, Metrics2] =
      ZLayer.fromFunction[Has[(Counter, Histogram)], Metrics2.Service](
        tup => new Service {
            def getRegistry(): Task[CollectorRegistry] =
              Task.effect(CollectorRegistry.defaultRegistry)

            def inc(tags: Array[String]): zio.Task[Unit] =
              inc(1.0, tags)

            def inc(amount: Double, tags: Array[String]): Task[Unit] =
              Task(tup.get._1.labels(tags:_*)inc(amount))

            def time(f: () => Unit, tags: Array[String]): Task[Double] = Task {
              val t = tup.get._2.labels(tags: _*).startTimer()
              f()
              t.observeDuration()
            }
          }
      )
}

In [ ]:
CollectorRegistry.defaultRegistry.clear()

Both `receiver` and `receiverHas` takes a Prometheus Counter and a Histogram as input:

In [ ]:
val c = Counter
    .build()
    .name("PrometheusCounter")
    .labelNames(Array("class", "method"): _*)
    .help(s"Sample prometheus counter")
    .register()

val h = Histogram
    .build()
    .name("PrometheusHistogram")
    .labelNames(Array("class", "method"): _*)
    .linearBuckets(0.0, 0.2, 5)
    .help(s"Sample prometheus histogram")
    .register()

The biggest difference is on how we pass them to each layer, `receiverHas` requires a `Zlayer` that `Has` a tuple `(Counter, Histogram)`:

In [ ]:
val rLayerHas     = ZLayer.succeed[(Counter, Histogram)]((c, h)) >>> Metrics2.receiverHas
val rtReceiverhas = Runtime.unsafeFromLayer(rLayerHas ++ Console.live)

If `++` is **compose/and** then you can read `>>>` as **feed/into**.
`receiver` on the other hand is easier to implement, but you may not have a choice as to which method to use.

In [ ]:
val rLayer     = Metrics2.receiver(c, h)
val rtReceiver = Runtime.unsafeFromLayer(rLayer ++ Console.live)

And finally a simple program to test it:

In [ ]:
val program2 = for {
      m  <- RIO.environment[Metrics2]
      _  <- putStrLn("Metrics2")
      _  <- m.get.inc(Array("RequestCounter", "get"))
      _  <- m.get.inc(Array("RequestCounter", "post"))
      _  <- m.get.inc(2.0, Array("LoginCounter", "login"))
      _  <- m.get.time(() => Thread.sleep(2000), Array("histogram", "get"))
      r  <- m.get.getRegistry()
      s  <- writeT004(r)
      _  <- putStrLn(s)
    } yield ()

rtReceiverhas.unsafeRun(program2)
//rtReceiver.unsafeRun(program2)

<a id="orge161833"></a>

# zio-test


<a id="org47c1454"></a>

## Intro

* `zio-test` is designed so that tests (and other accompanying concepts like assertions) become ordinary values that can be passed around, transformed and composed together.
* Testing asynchronous code is easy since instead of wrapping code in `scala.util.Future` (since you cannot block on ScalaJS), everything is already a `ZIO` object.
* Since test are ordinary `ZIO` values, we don&rsquo;t need to turn to testing framework for things like retries, timeouts and resource management.
* `Spec[L, T]`, every spec is labeled with `L` and can be a **suite** which contains other specs or a test of type `T`.

<a id="org3275b26"></a>

## TestClock

In [ ]:
import $ivy.`dev.zio::zio-test:1.0.0-RC18-2`

In [ ]:
import zio.duration._
import zio.{RIO, Schedule, Queue, UIO, URIO}
import zio.console._
import zio.test._
import zio.test.Assertion._
import zio.test.environment.TestClock

object ClockTest extends DefaultRunnableSpec {

    val queue: UIO[Queue[String]] = Queue.bounded[String](10)

    def sendAsync(msg: String)(q: Queue[String]): UIO[Unit] =
      for {
        _ <- q.offer(msg).fork
      } yield ()

    def listen(q: Queue[String]): RIO[Console, List[String]] = {
      import scala.collection.mutable.ListBuffer
      val collector = new ListBuffer[String]()
      for {
          s <- q.poll
          _ <- putStrLn(s"Received $s")
          if (s.isDefined)
      } yield {
          if (collector.size == 5) collector.clear()
          (collector += s.get).toList
      }
    }

    val every5Min = Schedule.spaced(1.second)
    val until5Collected = Schedule.doUntil[List[String]](_.size == 5)
    val every5SecUntilCollected = until5Collected && every5Min

    val val2Test = {
        for {
            q <- queue
            _ <- URIO.foreach(1 to 20)(i => sendAsync(s"Number $i")(q))
            l <- listen(q).repeat(every5SecUntilCollected)
        } yield l
    }

    def spec = suite("Testing Spec")(
        test("test simple values") {
            assert(5*5)(equalTo(25))
        },
        testM("test Monadic/ZIO values") {
            for {
                _   <- TestClock.adjust(5.seconds)
                tup <- val2Test
            } yield assert(tup._1.size)(equalTo(5)) && assert(tup._2)(equalTo(4))
        }
    )
}

ClockTest.main(Array.empty[String])

<a id="org6360a37"></a>

# ZIO Lock-in?

* Nope, `iterop` everything
* but&#x2026; zio-all is comfy

